In [ ]:
import json
import base64

In [ ]:
proj = 'nb'
nb = 'NotebookIn.ipynb'

In [ ]:
! aws codecommit delete-repository --repository-name nb

In [ ]:
%%bash -s "$proj"  --out output
aws codecommit create-repository \
--repository-name  $1

In [ ]:
repo = json.loads(output)
repo

In [ ]:
repo_url = repo['repositoryMetadata']['cloneUrlHttp']
repo_url

In [ ]:
%%bash -s "$nb" "$repo_url"
rm -rf .git 
git init
git add $1
git commit -m 'first'
git remote add origin $2
git push origin master


In [ ]:
repo_url = 'https://github.com/githubmg/ppmilltest.git'

In [ ]:
script = \
'sudo python -m ipykernel install --name conda_python3 \n' + \
'pip install papermill \n' + \
'git clone ' + repo_url + ' \n' + \
'python - << EOF \n' + \
'import papermill as pm \n' + \
'pm.execute_notebook( \n' + \
'   "/home/ec2-user/ppmilltest/' + nb +'", \n' + \
'   "/home/ec2-user/ppmilltest/' + nb[:-6] + 'out.ipynb" \n' + \
') \n' +  \
'EOF'

In [ ]:
encoded = base64.b64encode(script)

In [ ]:
encoded

In [ ]:
#! aws sagemaker   delete-notebook-instance-lifecycle-config --notebook-instance-lifecycle-config-name nb

In [ ]:
%%bash -s "$proj" "$encoded"
aws sagemaker create-notebook-instance-lifecycle-config \
--notebook-instance-lifecycle-config-name $1 \
--on-create '[{"Content": "c3VkbyBweXRob24gLW0gaXB5a2VybmVsIGluc3RhbGwgLS1uYW1lIGNvbmRhX3B5dGhvbjMgCnBpcCBpbnN0YWxsIHBhcGVybWlsbCAKZ2l0IGNsb25lIGh0dHBzOi8vZ2l0aHViLmNvbS9naXRodWJtZy9wcG1pbGx0ZXN0LmdpdCAKcHl0aG9uIC0gPDwgRU9GIAppbXBvcnQgcGFwZXJtaWxsIGFzIHBtIApwbS5leGVjdXRlX25vdGVib29rKCAKICAgIi9ob21lL2VjMi11c2VyL3BwbWlsbHRlc3QvTm90ZWJvb2tJbi5pcHluYiIsIAogICAiL2hvbWUvZWMyLXVzZXIvcHBtaWxsdGVzdC9Ob3RlYm9va0lub3V0LmlweW5iIiAKKSAKRU9G"}]'

In [ ]:
%%bash -s "$proj" "$repo_url"
aws sagemaker create-notebook-instance \
--notebook-instance-name $1 \
--instance-type ml.t2.medium \
--role-arn arn:aws:iam::452432741922:role/service-role/AmazonSageMaker-ExecutionRole-20181022T121720 \
--lifecycle-config-name $1 

In [ ]:
%%bash
aws sagemaker describe-notebook-instance --notebook-instance-name nb